# Using FFT to discover the right spectrum of the incoming signal 


In [1]:
%pylab inline
# import numpy as np
# from scipy import fft
from __future__ import division

# redefine default figure size and fonts
import matplotlib as mpl
mpl.rc('text', usetex = True)
mpl.rc('font', family = 'serif',size=14)
mpl.rc('figure',figsize=(10,8))



Populating the interactive namespace from numpy and matplotlib


## Two basic ploting functions that repeat the actuall spectral analysis:

In [2]:
def spectrum(y,Fs):
    """
    Plots a Single-Sided Amplitude Spectrum of a sampled
    signal y(t), sampling frequency Fs (lenght of a signal 
    provides the number of samples recorded)
    
    Following: http://goo.gl/wRoUn
    """
    n = len(y) # length of the signal
    k = arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[range(np.int(n/2))] # one side frequency range
    Y = 2*fft(y)/n # fft computing and normalization
    Y = Y[range(np.int(n/2))]
    return (frq, Y)

# import signal generator
from create_signal import create_signal

def plotSignal(fs,N):
    """ plots the time signal Y(t) and the 
    frequency spectrum Y(fs)
    Inputs:
        fs - sampling frequency in Hz
        N  - total number of sample points, length of a sample
    Outputs:
        t - time signal, [sec]
        Y - values, [Volt]
    Usage:
        plotSignal(fs=30,N=256)
    """
    t,y = create_signal(fs,N)
    y = y - np.mean(y)
    frq,Y = spectrum(y,fs) 
    
    # Plot
    figure()
    subplot(2,1,1)
    plot(t,y,'b-')
    xlabel('$t$ [s]')
    ylabel('$Y$ [V]')
    # axes().set_aspect(0.2)
    # title('sampled signal')
    subplot(2,1,2)
    plot(frq,abs(Y),'r') # plotting the spectrum
    xlabel('$f$ (Hz)')
    ylabel('$|Y(f)|$')
    
    return t,y

In [3]:
# %loadpy create_signal

In [4]:
from numpy.random import normal
# create signal
def create_signal(fs,N):
    """ create signal with gaussian noise"""
    dt = 1/fs
    t = np.linspace(0,N*dt,N)
    # print t
    y = 1.0*np.sin(2*np.pi*10*t) + 1.5*np.sin(2*np.pi*35.7*t)
    # print y
    noise = normal(0,1,N)
    # print noise
    sig=y+noise
    # print sig
    return t,sig



## Let's sample 256 points at 30 Hz

In [5]:
fs = 30 # sampling frequency, Hz
N = 256 # number of sampling points
plotSignal(fs,N)
t,y = create_signal(fs,N)
# sampling time
T = t[-1] - t[0]
print 'Sampling time T =%6.3f sec' % T
# Number of points
N = len(t)
print 'Number of samples N = %d' % N
# sampling frequency 
fs  = N/T
print 'Sampling frequency fs = %6.3f Hz' % fs

TypeError: 'module' object is not callable

## Sample at higher fs, but not INTERGER times higher

In [ ]:
plotSignal(66,256)

In [ ]:
fs = 66; N = 256
t,y = create_signal(fs,N)
frq,Y = spectrum(y,fs)
max(abs(Y)), frq[argmax(abs(Y))]

## There is a problem with the previous sampling 

The 5.7 Hz peak is replaced now by approximately 30 Hz peak. We sampled at 66 Hz, so 30 Hz is below Nyquist and all seems to be fine
But we know that if the right frequency is 30.3 or 30.15 Hz, then when we sampled at 30 Hz we should get: |30.3 - 30] = .3 Hz.
We got 5.7 Hz, so it cannot be that 30.3 is correct, or at least we have some discrepancy here. 
Let's do get even higher sampling frequency: fs = 100 Hz it is not X times 66 

In [ ]:
fs = 100; N = 256
t,y = create_signal(fs,N)
frq,Y = spectrum(y,fs)
plotSignal(fs,N)
print max(abs(Y[1:])), frq[argmax(abs(Y[1:]))]


## So the problem is still there:
the peak is now shifted to 35.5 Hz, let's see: when we sampled at 30 we got aliased 35.5 - 30 = 5.5 Hz. Seems correct,
when we sampled at 66 it was above 2/3*f but below 2*f, so 66 - 35.5 = 30.5 Hz, also seem to fit. So now we have explained (!)
the previous results and we believe that hte signal has 10 Hz and 35.5 Hz (approximately) and 100 Hz is good enough
but we could improve it by going slightly above Nyquist 2*35.5 = 71 Hz, and also try to fix the number of points such that
we get integer number of periods. It's imposible since we have two frequencies, but at least we can try to get it close
enough: 1/10 = 0.1 sec so we could do 256 * 0.1 = 25.6 Hz or X times that = 25.6 * 3 = 102.4 - could be too close
let's try:

In [ ]:
plotSignal(102.4,256)

Then the same principle to get better 35.5 Hz
1/35.5 * 256 = 7.21 * 15 = 108.16 Hz

In [ ]:
plotSignal(108.16,256)

In [ ]:
plotSignal(108.16,256)

In [ ]:
# 1/35.5 * 256 * 20 = 144.22
plotSignal(144.22,512)

In [ ]:
## Why not to start from high frequencies:

In [ ]:
plotSignal(fs=500,N=256)

* Very poor frequency resolution, $\Delta f = 1/T = f_s/N = 500 / 256 = 1.95 Hz$ versus $\Delta f = 100 / 256 = 0.39 Hz$
* peaks are at 9.76 Hz and 35.2 Hz 
* wasting a lot of frequency axis above 2 * 35.7

## Use windowing or low-pass filtering
    

In [ ]:
from scipy.signal import hann
h = hann(N)

In [ ]:
fs = 150; N = 256;
t,y = create_signal(fs,N)

plot(t,y,'b-',t,y*h,'r')

In [ ]:

frq,Y = spectrum(y,fs)
plot(frq,abs(Y))

## Oh, no, we forgot the DC  see the Y(0) !!

In [ ]:
frq,Y = spectrum(y-mean(y),fs)
plot(frq,abs(Y))

In [ ]:
from scipy.signal import firwin, lfilter
a = 1                                          # filter denominator
b = firwin(N, cutoff=.6, window='hamming')    # filter numerator
y1 = lfilter(b, a, y-mean(y)) 
plot(t,y,'b',t,y1+mean(y),'r')

In [ ]:
frq1,Y1 = spectrum(y1,fs)

In [ ]:
plot(frq,abs(Y),'b',frq1,abs(Y1),'r')